# AutoML in pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a pipeline with Regression AutoML task.

**Motivations** - This notebook explains how to use Regression AutoML task inside pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, command, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.automl import classification, regression
from azure.ai.ml.entities._job.automl.tabular import TabularFeaturizationSettings
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    BatchEndpoint, ModelBatchDeployment, ModelBatchDeploymentSettings
)

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

## 2. Create Deployment Targets (Batch/Real-Time Endpoints)

### 2.1 Create Managed Batch Endpoint

In [ ]:
# Define an endpoint name
endpoint_name = "battery-rul-prediction-batch"

# create an online endpoint
endpoint = BatchEndpoint(
    name = endpoint_name, 
    description="Batch endpoint for predicting remaining useful battery life based on cycle telemetry"
)

ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

### 2.2 Create Managed Real-Time Endpoint

In [ ]:
# Define an endpoint name
endpoint_name = "battery-rul-prediction-rt"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="Real-time HTTP endpoint for predicting remaining useful battery life based on cycle telemetry",
    auth_mode="key"
)

ml_client.online_endpoints.begin_create_or_update(endpoint).result()

# 3. Basic pipeline job with regression task

## 3.1 Build pipeline

In [ ]:
preprocessing_env = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="./environment/preprocessing_env.yaml",
    name="pipeline-custom-environment",
    description="Preprocessing environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(preprocessing_env)


evaluation_env = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="./environment/evaluation_env.yaml",
    name="pipeline-evaluation-environment",
    description="Evaluation environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(evaluation_env)

deployment_env = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="./environment/deployment_env.yaml",
    name="pipeline-deployment-environment",
    description="Deployment environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(deployment_env)

battery_model_env = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="./environment/reusable_model_env.yaml",
    name="battery-model-deployment-environment",
    description="Model deployment environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(battery_model_env)

In [ ]:
# Define pipeline
@pipeline(
    description="AutoML Regression Pipeline",
)
def automl_regression(
    regression_train_data, regression_validation_data, regression_test_data, model_base_name
):
    
    # define command function for preprocessing the model
    preprocessing_command_func = command(
        inputs=dict(
            delta_table='abfss://mldata@honadlsgen2demo.dfs.core.windows.net/battery_cycle_rul_data'
        ),
        outputs=dict(
            preprocessed_train_data=Output(type="mltable"),
            preprocessed_test_data=Output(type="mltable"),
        ),
        code="./preprocess.py",
        command="python preprocess.py "
        + "--delta_table ${{inputs.delta_table}} "
        + "--preprocessed_train_data ${{outputs.preprocessed_train_data}} "
        + "--preprocessed_test_data ${{outputs.preprocessed_test_data}}",
        environment="pipeline-custom-environment@latest",
        display_name='Get and Split Data'
    )
    preprocess_node = preprocessing_command_func()

    # define the AutoML regression task with AutoML function
    regression_node = regression(
        primary_metric="r2_score",
        target_column_name="RUL",
        training_data=preprocess_node.outputs.preprocessed_train_data,
        test_data=preprocess_node.outputs.preprocessed_test_data,
        featurization=TabularFeaturizationSettings(mode="auto"),
        # currently need to specify outputs "mlflow_model" explicitly to reference it in following nodes
        outputs={"best_model": Output(type="mlflow_model")},
        display_name='Train Models'
    )
    # set limits & training
    regression_node.set_limits(max_trials=20, max_concurrent_trials=4)
    regression_node.set_training(
        enable_stack_ensemble=False, enable_vote_ensemble=False, enable_model_explainability=True
    )
    
    # define command function for evaluating the newly trained model (champion v. challenger test)
    evaluate_func = command(
        inputs=dict(
            model_input_path=Input(type="mlflow_model"),
            model_base_name='Battery_Cycle_RUL_Prediction_Model',
            test_data=Input(type="mltable"),
            target_column='RUL'
        ),
        outputs=dict(
             comparison_metrics=Output(type="uri_folder"),
        ),
        code="./evaluate.py",
        command="python evaluate.py "
        + "--model_input_path ${{inputs.model_input_path}} "
        + "--model_base_name ${{inputs.model_base_name}} "
        + "--test_data ${{inputs.test_data}} "
        + "--target_column ${{inputs.target_column}} "
        + "--comparison_metrics ${{outputs.comparison_metrics}}",
        environment="pipeline-evaluation-environment@latest",
        display_name='Evaluate Model (Champion vs. Challenger)'
    )
    evaluate_model = evaluate_func(test_data=preprocess_node.outputs.preprocessed_test_data, model_input_path=regression_node.outputs.best_model)

    # define command function for registering the model
    command_func = command(
        inputs=dict(
            model_input_path=Input(type="mlflow_model"),
            model_base_name='Battery_Cycle_RUL_Prediction_Model',
            comparison_metrics=Input(type="uri_folder"),
        ),
        outputs=dict(
            registered_model_details=Output(type="uri_folder")
        ),
        code="./register.py",
        command="python register.py "
        + "--model_input_path ${{inputs.model_input_path}} "
        + "--model_base_name ${{inputs.model_base_name}} "
        + "--registered_model_details ${{outputs.registered_model_details}} "
        + "--comparison_metrics ${{inputs.comparison_metrics}} ",
        environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1",
        display_name='Register Model'
    )
    register_model = command_func(model_input_path=regression_node.outputs.best_model, comparison_metrics=evaluate_model.outputs.comparison_metrics)

    deploy_batch_func = command(
        inputs=dict(
            model_base_name='Battery_Cycle_RUL_Prediction_Model',
            endpoint_name='battery-rul-prediction-batch',
            registered_model_details=Input(type="uri_folder")
        ),
        code="./deploy_batch.py",
        command="python deploy_batch.py "
        + "--model_base_name ${{inputs.model_base_name}} "
        + "--registered_model_details ${{inputs.registered_model_details}} "
        + "--endpoint_name ${{inputs.endpoint_name}} ",
        environment="pipeline-deployment-environment@latest",
        display_name='Update Batch Deployment'
    )
    deploy_batch = deploy_batch_func(registered_model_details=register_model.outputs.registered_model_details)
    
    
    deploy_rt_func = command(
        inputs=dict(
            model_base_name='Battery_Cycle_RUL_Prediction_Model',
            endpoint_name='battery-rul-prediction-rt',
            registered_model_details=Input(type="uri_folder")
        ),
        code="./deploy_real_time.py",
        command="python deploy_real_time.py "
        + "--model_base_name ${{inputs.model_base_name}} "
        + "--registered_model_details ${{inputs.registered_model_details}} "
        + "--endpoint_name ${{inputs.endpoint_name}} ",
        environment="pipeline-deployment-environment@latest",
        display_name='Update Real-Time Deployment'
    )
    deploy_rt = deploy_rt_func(registered_model_details=register_model.outputs.registered_model_details)


pipeline_regression = automl_regression()

# set pipeline level compute
pipeline_regression.settings.default_compute = "cpu-cluster"

# 3.2 Submit pipeline job

In [ ]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_regression, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

## 4. Testing

### 4.1 Invoke real-time endpoint

Run this cell after your pipeline execution has completed

In [ ]:
import requests
import json

rt_endpoint_name = 'battery-rul-prediction-rt'
rt_endpoint = ml_client.online_endpoints.get(rt_endpoint_name)
scoring_uri = rt_endpoint.scoring_uri
auth_key = ml_client.online_endpoints.get_keys(name=rt_endpoint_name).primary_key

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ auth_key)}
sample_data = {
  "input_data": {
    "columns": [
      'Discharge Time (s)', 'Decrement 3.6-3.4V (s)', 'Max. Voltage Dischar. (V)', 'Min. Voltage Charg. (V)', 'Time at 4.15V (s)', 'Time constant current (s)', 'Charging time (s)', 'Total time (s)'
    ],
    "data": [[2652.99, 1364.991, 3.652, 3.233, 5340.993, 6664, 11018.02, 14949.34]]
  }
}

resp = requests.post(scoring_uri, headers=headers, data=json.dumps(sample_data))
resp.json()

# Next Steps
You can see further examples of running a pipeline job [here](../)

- [Invoke your batch endpoint and access results](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-batch-model-deployments?view=azureml-api-2&tabs=python#run-batch-endpoints-and-access-results)
- [Perform continuous monitoring of models in production](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-monitor-model-performance?view=azureml-api-2&tabs=python)